<a href="https://colab.research.google.com/github/HesusG/diagnostico-lineas-accion/blob/main/Semana2/ejercicios_extra/practica_ong_estadistica_avanzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica: Fundación Esperanza - Estadística Avanzada

**Objetivo:** Aplicar Chi-cuadrada, ANOVA y Regresión Lineal al análisis de retención de donadores.

**Preguntas de negocio:**
1. ¿El tipo de donador está relacionado con la frecuencia de donación? (Chi-cuadrada)
2. ¿La satisfacción varía según el canal de donación? (ANOVA)
3. ¿Los años como donador predicen el compromiso futuro? (Regresión)

---

## Parte 0: Configuración

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
%matplotlib inline

In [ ]:
# Cargar dataset
df = pd.read_csv('https://raw.githubusercontent.com/HesusG/diagnostico-lineas-accion/main/Semana1/datos/fundacion_esperanza_donadores.csv')

print(f"Dataset: {df.shape[0]} donadores × {df.shape[1]} variables")
df.head()

In [ ]:
# Variables de interés
vars_interes = ['tipo_donante', 'frecuencia_donacion', 'canal_donacion', 
                'años_como_donante', 'satisfaccion_general', 'compromiso_futuro']

print("Información de variables:")
print(df[vars_interes].info())
print("\nValores faltantes:")
print(df[vars_interes].isnull().sum())

---

## Parte 1: Chi-Cuadrada (χ²)

**Pregunta:** ¿El tipo de donador está relacionado con la frecuencia de donación?

### 1.1 Hipótesis

- **H₀:** [TU RESPUESTA]
- **H₁:** [TU RESPUESTA]
- **α = 0.05**

### 1.2 Tabla de Contingencia

In [ ]:
# TU CÓDIGO: Crear tabla de contingencia
tabla_contingencia = pd.crosstab(df['tipo_donante'], df['frecuencia_donacion'])
print(tabla_contingencia)

In [ ]:
# Visualización
sns.heatmap(tabla_contingencia, annot=True, fmt='d', cmap='YlOrRd')
plt.title('Tipo de Donante × Frecuencia de Donación')
plt.tight_layout()
plt.show()

**Observaciones:** [TU RESPUESTA]

### 1.3 Prueba Chi-Cuadrada

In [ ]:
# TU CÓDIGO: Ejecutar prueba
chi2_stat, p_value, dof, expected_freq = chi2_contingency(tabla_contingencia)

print(f"χ² = {chi2_stat:.4f}")
print(f"p-value = {p_value:.4f}")
print(f"Grados de libertad = {dof}")

In [ ]:
# Residuos estandarizados
expected_df = pd.DataFrame(expected_freq, index=tabla_contingencia.index, columns=tabla_contingencia.columns)
residuos = (tabla_contingencia - expected_df) / np.sqrt(expected_df)

sns.heatmap(residuos, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Residuos Estandarizados (|valor| > 2 indica asociación)')
plt.tight_layout()
plt.show()

### 1.4 Interpretación

**a) Decisión:** [TU RESPUESTA]

**b) Patrones en residuos:** [TU RESPUESTA]

**c) Implicaciones estratégicas:** [TU RESPUESTA]

---

## Parte 2: ANOVA

**Pregunta:** ¿La satisfacción varía según el canal de donación?

### 2.1 Hipótesis

- **H₀:** [TU RESPUESTA]
- **H₁:** [TU RESPUESTA]
- **α = 0.05**

### 2.2 Estadísticas Descriptivas

In [ ]:
# TU CÓDIGO: Estadísticas por canal
stats_canal = df.groupby('canal_donacion')['satisfaccion_general'].agg(['count', 'mean', 'std', 'min', 'max'])
print(stats_canal)

In [ ]:
# Visualización
sns.boxplot(data=df, x='canal_donacion', y='satisfaccion_general')
plt.axhline(df['satisfaccion_general'].mean(), color='red', linestyle='--', label='Media global')
plt.title('Satisfacción por Canal de Donación')
plt.legend()
plt.tight_layout()
plt.show()

**Observaciones:** [TU RESPUESTA]

### 2.3 Prueba ANOVA

In [ ]:
# TU CÓDIGO: Separar grupos y ejecutar ANOVA
grupos = [df[df['canal_donacion'] == canal]['satisfaccion_general'] for canal in df['canal_donacion'].unique()]
f_stat, p_value_anova = f_oneway(*grupos)

print(f"F = {f_stat:.4f}")
print(f"p-value = {p_value_anova:.4f}")

### 2.4 Prueba Post-Hoc (Tukey)

In [ ]:
# TU CÓDIGO: Tukey HSD
tukey_result = pairwise_tukeyhsd(df['satisfaccion_general'], df['canal_donacion'], alpha=0.05)
print(tukey_result)

### 2.5 Interpretación

**a) Decisión ANOVA:** [TU RESPUESTA]

**b) Pares significativamente diferentes (Tukey):** [TU RESPUESTA]

**c) Ranking de canales:** [TU RESPUESTA]

**d) Recomendaciones:** [TU RESPUESTA]

---

## Parte 3: Regresión Lineal

**Pregunta:** ¿Los años como donador predicen el compromiso futuro?

### 3.1 Exploración

In [ ]:
# Scatter plot
plt.scatter(df['años_como_donante'], df['compromiso_futuro'], alpha=0.5)
plt.xlabel('Años como Donante')
plt.ylabel('Compromiso Futuro')
plt.title('Años vs Compromiso Futuro')
plt.grid(alpha=0.3)
plt.show()

**Observaciones:** [TU RESPUESTA]

### 3.2 Correlación de Pearson

In [ ]:
# TU CÓDIGO: Calcular correlación
r_pearson, p_value_corr = stats.pearsonr(df['años_como_donante'], df['compromiso_futuro'])

print(f"r = {r_pearson:.4f}")
print(f"p-value = {p_value_corr:.4f}")

**Interpretación:** [TU RESPUESTA]

### 3.3 Modelo de Regresión

In [ ]:
# TU CÓDIGO: Ajustar modelo
X = sm.add_constant(df['años_como_donante'])
y = df['compromiso_futuro']
modelo = sm.OLS(y, X).fit()

print(modelo.summary())

### 3.4 Parámetros del Modelo

In [ ]:
intercepto = modelo.params['const']
pendiente = modelo.params['años_como_donante']
r_cuadrado = modelo.rsquared

print(f"Ecuación: compromiso = {intercepto:.3f} + {pendiente:.4f} × años")
print(f"R² = {r_cuadrado:.4f}")

**Interpretación:**

**a) Intercepto:** [TU RESPUESTA]

**b) Pendiente:** [TU RESPUESTA]

**c) R²:** [TU RESPUESTA]

### 3.5 Visualización

In [ ]:
# Gráfico con línea de regresión
plt.scatter(df['años_como_donante'], df['compromiso_futuro'], alpha=0.5, label='Datos')

x_line = np.linspace(df['años_como_donante'].min(), df['años_como_donante'].max(), 100)
y_line = intercepto + pendiente * x_line
plt.plot(x_line, y_line, 'r-', linewidth=2, label=f'Regresión (R²={r_cuadrado:.3f})')

plt.xlabel('Años como Donante')
plt.ylabel('Compromiso Futuro')
plt.title('Modelo de Regresión Lineal')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

### 3.6 Predicciones

In [ ]:
# Predicciones para diferentes años
años_pred = [0, 5, 10, 15, 20]

for años in años_pred:
    compromiso_pred = intercepto + pendiente * años
    print(f"Años: {años:2d} → Compromiso predicho: {compromiso_pred:.2f}")

**Análisis de predicciones:** [TU RESPUESTA]

---

## Parte 4: Reflexión MEAL Comparativa (Semanas 1 y 2)

**Estructura:** 300-400 palabras integrando hallazgos de ambas semanas

**M - Main Idea (50-75 palabras):**

[TU RESPUESTA]

---

**E - Evidence (100-125 palabras):**

[TU RESPUESTA]

---

**A - Analysis (100-125 palabras, mínimo 2 citas APA):**

[TU RESPUESTA]

---

**L - Link (50-75 palabras):**

[TU RESPUESTA]

---

**Referencias:**

[Agrega tus referencias en formato APA 7]